In [1]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder


#enter your API key, you can make one for free on HF
notebook_login()

Login successful
Your token has been saved to C:\Users\felip/.huggingface/token


In [28]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [29]:
import time
def bloom(prompt,
          max_length = 32,
          top_k = 0,
          num_beams = 0,
          no_repeat_ngram_size = 2,
          top_p = 0.9,
          seed=42,
          temperature=0.7,
          greedy_decoding = False,
          return_full_text = False):
    
    top_k = None if top_k == 0 else top_k
    do_sample = False if num_beams > 0 else not greedy_decoding
    num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    top_p = None if num_beams else top_p
    early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": max_length,
        "top_k": top_k,
        "top_p": top_p,
        "temperature": temperature,
        "do_sample": do_sample,
        "seed": seed,
        "early_stopping":early_stopping,
        "no_repeat_ngram_size":no_repeat_ngram_size,
        "num_beams":num_beams,
        "return_full_text":return_full_text
    }
    try:
        s = time.time()
        response = inference(prompt, params=params)
        #print(response)
        proc_time = time.time()-s
        print(f"Processing time was {proc_time} seconds")
    except:
        print("Thinking...")
        return bloom(prompt,
          max_length = max_length,
          top_k = top_k,
          num_beams = num_beams,
          no_repeat_ngram_size = no_repeat_ngram_size,
          top_p = top_p,
          seed=seed,
          temperature=temperature,
          greedy_decoding = greedy_decoding,
          return_full_text = return_full_text)
    return response

In [30]:
zero_shot_prompt = lambda q, a: f"""Esto es un clasificador de incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. 
Daremos preguntas y respuestas. 
Luego se determinara si cada respuesta es coherente, o bien incoherente, a su pregunta.
Marcaremos las preguntas entre <P y /P>, y las respuestas entre <R {{respuesta}} /R>.
Se entregaran las etiquetas incoherentes por <E Incoherente /E>, o bien coherentes por <E Coherente /E>, segun corresponda.
Siempre daremos un etiqueta y terminaremos la pagina con /E>.

<P {q} /P>. 
<R {a} /R>.
<E """

q = "Antonella compró 23 dulces para su curso. Regaló algunos dulces a sus compañeros y le quedan 9 en su bolsillo. Una amiga le dijo que regaló 11 dulces en total.¿ Es correcto lo que señala su amiga? Explica con tus palabras."
a = "no es verdad porque reste y me dio 14"
zero_shot_prompt(q, a)

'Esto es un clasificador de incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nDaremos preguntas y respuestas. \nLuego se determinara si cada respuesta es coherente, o bien incoherente, a su pregunta.\nMarcaremos las preguntas entre <P y /P>, y las respuestas entre <R {respuesta} /R>.\nSe entregaran las etiquetas incoherentes por <E Incoherente /E>, o bien coherentes por <E Coherente /E>, segun corresponda.\nSiempre daremos un etiqueta y terminaremos la pagina con /E>.\n\n<P Antonella compró 23 dulces para su curso. Regaló algunos dulces a sus compañeros y le quedan 9 en su bolsillo. Una amiga le dijo que regaló 11 dulces en total.¿ Es correcto lo que señala su amiga? Explica con tus palabras. /P>. \n<R no es verdad porque reste y me dio 14 /R>.\n<E '

In [31]:
q, a = test.iloc[120]["Q A".split()]
q, a

(' Pedro tiene un chocolate que está dividido en cuatro partes iguales y se comió un trozo.¿Qué fracción del total del chocolate se comió Pedro?',
 'pedro comio 1 troso del chocolate y le quedo 1/4\n')

In [32]:
bloom(zero_shot_prompt(str(q).replace("\n", ""), str(a).replace("\n", "")), max_length=7, temperature=0.4)#[0]['generated_text']

Processing time was 0.8786895275115967 seconds


{'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}

In [6]:
import pandas as pd

In [7]:
test = pd.read_excel("data/test_task_C1.xlsx")

In [130]:
%%time
# responses = []
for i, ix in enumerate(test.index[len(responses):]):
    q, a = test.loc[ix]["Q A".split()]
    o = bloom(zero_shot_prompt(str(q).replace("\n", ""), str(a).replace("\n", "")), max_length=7, temperature=0.4)[0]['generated_text']
    responses.append([ix, o])
    print(100*(i+1)/test.shape[0], ix, o)

Processing time was 3.9767346382141113 seconds
0.14771048744460857 419  Incoherente /E>
Processing time was 1.1559886932373047 seconds
0.29542097488921715 420  Incoherente /E>
Processing time was 2.3065648078918457 seconds
0.4431314623338257 421  Incoherente /E>
Processing time was 2.944833993911743 seconds
0.5908419497784343 422  Incoherente /E>
Processing time was 1.9283397197723389 seconds
0.7385524372230429 423  Incoherente /E>
Processing time was 2.265040874481201 seconds
0.8862629246676514 424  Incoherente /E>
Processing time was 2.4544873237609863 seconds
1.03397341211226 425  Incoherente /E>
Processing time was 2.459059715270996 seconds
1.1816838995568686 426  Incoherente /E>
Processing time was 2.674804210662842 seconds
1.329394387001477 427  Incoherente /E>
Processing time was 1.9585647583007812 seconds
1.4771048744460857 428  Incoherente /E>
Processing time was 2.7537670135498047 seconds
1.6248153618906942 429  Incoherente /E>
Processing time was 3.077760696411133 seconds
1.

Processing time was 3.2387475967407227 seconds
14.32791728212703 515  Incoherente /E>
Processing time was 3.0034496784210205 seconds
14.47562776957164 516  Incoherente /E>
Processing time was 1.7807323932647705 seconds
14.623338257016249 517  Incoherente /E>
Processing time was 2.9572792053222656 seconds
14.771048744460856 518  Incoherente /E>
Processing time was 2.6431796550750732 seconds
14.918759231905465 519  Incoherente /E>
Processing time was 3.391906499862671 seconds
15.066469719350074 520  Incoherente /E>
Processing time was 2.175935983657837 seconds
15.214180206794682 521  Incoherente /E>
Processing time was 3.517761707305908 seconds
15.36189069423929 522  Incoherente /E>
Processing time was 2.969796657562256 seconds
15.5096011816839 523  Incoherente /E>
Processing time was 1.7765426635742188 seconds
15.657311669128507 524  Incoherente /E>
Processing time was 1.9539976119995117 seconds
15.805022156573116 525  Incoherente /E>
Processing time was 2.805767297744751 seconds
15.952

Processing time was 3.3361337184906006 seconds
28.50812407680945 611  Incoherente /E>
Processing time was 2.8109586238861084 seconds
28.65583456425406 612  Incoherente /E>
Processing time was 1.9434945583343506 seconds
28.80354505169867 613  Incoherente /E>
Processing time was 3.0029351711273193 seconds
28.95125553914328 614  Incoherente /E>
Processing time was 2.514010429382324 seconds
29.098966026587888 615  Incoherente /E>
Processing time was 3.155285120010376 seconds
29.246676514032497 616  Incoherente /E>
Processing time was 2.7541754245758057 seconds
29.394387001477106 617  Incoherente /E>
Processing time was 2.1674911975860596 seconds
29.542097488921712 618  Incoherente /E>
Processing time was 3.112706184387207 seconds
29.68980797636632 619  Incoherente /E>
Processing time was 2.386829137802124 seconds
29.83751846381093 620  Incoherente /E>
Processing time was 2.7435145378112793 seconds
29.98522895125554 621  Incoherente /E>
Processing time was 2.0959677696228027 seconds
30.1329

In [132]:
test_v1 = test.copy()
test_v1["bloom"] = [x[1] for x in responses]
test_v1["bloom-label"] = [1 if "incoherente" in x[1].lower() else 0 for x in responses]
test_v1

,id,Q_id,Q,A,label,bloom,bloom-label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,Incoherente /E>,1
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Incoherente /E>,1
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,Incoherente /E>,1
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,Incoherente /E>,1
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,Incoherente /E>,1
...,...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,Incoherente /E>,1
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Incoherente /E>,1
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,Incoherente /E>,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,Incoherente /E>,1


In [133]:
test_v1.to_excel("bloom_label_v1.xlsx")

In [40]:
from sklearn.metrics import classification_report

In [135]:
print(classification_report(test_v1["label"], test_v1["bloom-label"], digits=4))

              precision    recall  f1-score   support

           0     0.5000    0.0111    0.0217       541
           1     0.1955    0.9559    0.3246       136

    accuracy                         0.2009       677
   macro avg     0.3477    0.4835    0.1731       677
weighted avg     0.4388    0.2009    0.0825       677



In [23]:
zero_shot_prompt_advice = lambda q, a: f"""Esto es un clasificador de incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. 
Daremos preguntas y respuestas. 
Luego se determinara si cada respuesta es coherente, o bien incoherente, a su pregunta.
Marcaremos las preguntas entre <P y /P>, y las respuestas entre <R {{respuesta}} /R>.
Se entregaran las etiquetas incoherentes por <E Incoherente /E>, o bien coherentes por <E Coherente /E>, segun corresponda.
Siempre daremos un etiqueta y terminaremos la pagina con /E>.
Iremos mas lento en los siguientes casos: 
Recursividad: Si preguntamos por P, pero P tiene varias preguntas P(Q1, Q2, etc) entonces debes distinguir entre P y las Q.
Correctitud: Las respuestas incorrectas pueden ser coherentes. 
<P {q} /P>. 
<R {a} /R>.
<E """

q = "Antonella compró 23 dulces para su curso. Regaló algunos dulces a sus compañeros y le quedan 9 en su bolsillo. Una amiga le dijo que regaló 11 dulces en total.¿ Es correcto lo que señala su amiga? Explica con tus palabras."
a = "no es verdad porque reste y me dio 14"
zero_shot_prompt_advice(q, a)

'Esto es un clasificador de incoherencia en respuestas a preguntas abiertas de alumnos de cuarto grado en matematicas. \nDaremos preguntas y respuestas. \nLuego se determinara si cada respuesta es coherente, o bien incoherente, a su pregunta.\nMarcaremos las preguntas entre <P y /P>, y las respuestas entre <R {respuesta} /R>.\nSe entregaran las etiquetas incoherentes por <E Incoherente /E>, o bien coherentes por <E Coherente /E>, segun corresponda.\nSiempre daremos un etiqueta y terminaremos la pagina con /E>.\nIremos mas lento en los siguientes casos: \nRecursividad: Si preguntamos por P, pero P tiene varias preguntas P(Q1, Q2, etc) entonces debes distinguir entre P y las Q.\nCorrectitud: Las respuestas incorrectas pueden ser coherentes. \n<P Antonella compró 23 dulces para su curso. Regaló algunos dulces a sus compañeros y le quedan 9 en su bolsillo. Una amiga le dijo que regaló 11 dulces en total.¿ Es correcto lo que señala su amiga? Explica con tus palabras. /P>. \n<R no es verdad 

In [35]:
%%time
# responses = []
for i, ix in enumerate(test.index[len(responses):]):
    q, a = test.loc[ix]["Q A".split()]
    o = bloom(zero_shot_prompt_advice(str(q).replace("\n", ""), str(a).replace("\n", "")), max_length=7, temperature=0.4)[0]['generated_text']
    responses.append([ix, o])
    print(100*(i+1)/test.shape[0], ix, o)

Processing time was 2.2864747047424316 seconds
0.14771048744460857 595  Incoherente /E>
Processing time was 1.1191632747650146 seconds
0.29542097488921715 596  Incoherente /E>
Processing time was 1.226973056793213 seconds
0.4431314623338257 597  Incoherente /E>
Processing time was 1.1540601253509521 seconds
0.5908419497784343 598  Incoherente /E>
Processing time was 2.401120901107788 seconds
0.7385524372230429 599  Incoherente /E>
Processing time was 1.1604821681976318 seconds
0.8862629246676514 600  Incoherente /E>
Processing time was 1.098686695098877 seconds
1.03397341211226 601  Incoherente /E>
Processing time was 1.0979950428009033 seconds
1.1816838995568686 602  Incoherente /E>
Processing time was 1.1807842254638672 seconds
1.329394387001477 603  Incoherente /E>
Processing time was 1.4210190773010254 seconds
1.4771048744460857 604  Incoherente /E>
Processing time was 1.222111701965332 seconds
1.6248153618906942 605  /E>.
<P Diego
Processing time was 1.4162890911102295 seconds
1.7

In [37]:
test_v2 = test.copy()
test_v2["bloom"] = [x[1] for x in responses]
test_v2["bloom-label"] = [1 if "incoherente" in x[1].lower() else 0 for x in responses]
test_v2

,id,Q_id,Q,A,label,bloom,bloom-label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,Incoherente /E>,1
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Incoherente /E>,1
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,Incoherente /E>,1
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,Incoherente /E>,1
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,Incoherente /E>,1
...,...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,Incoherente /E>,1
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Incoherente /E>,1
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,Incoherente /E>,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,Incoherente /E>,1


In [38]:
test_v2.to_excel("bloom_label_v2.xlsx")

In [41]:
print(classification_report(test_v2["label"], test_v2["bloom-label"], digits=4))

              precision    recall  f1-score   support

           0     0.7826    0.0665    0.1227       541
           1     0.1997    0.9265    0.3286       136

    accuracy                         0.2393       677
   macro avg     0.4911    0.4965    0.2256       677
weighted avg     0.6655    0.2393    0.1640       677

